In [19]:
from dotenv import find_dotenv, load_dotenv
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from typing import Optional
from pydantic import BaseModel, Field
load_dotenv(find_dotenv())

True

In [15]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [16]:
class OrderQuery(BaseModel):
    order_id: Optional [int]= Field(description='Unique order id of an Order')
    product_name: Optional [str] = Field(description='Name of Product')
    issue_type: str = Field(description='Return, Status, Recommendation')
    details: Optional[str] = Field(description='Order details.')



parser = PydanticOutputParser(pydantic_object= OrderQuery)
format_inst = parser.get_format_instructions()
format_inst

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"order_id": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "Unique order id of an Order", "title": "Order Id"}, "product_name": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Name of Product", "title": "Product Name"}, "issue_type": {"description": "Return, Status, Recommendation", "title": "Issue Type", "type": "string"}, "details": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Order details.", "title": "Details"}}, "required": ["order_id", "product_name", "issu

In [ ]:

user_query = PromptTemplate.from_template("""
You are a Polite Customer Executive, Your response to the customer's query is very polite, short and informative,
                                                       Answer the query in a format, here is your formatting instructions: {formatting_instructions} 
                                                       here is your  query: {query},
                                              Answer:
                                                       """, 
                                                       input_variable= ['query'],
                                                       partial_variables={'formatting_instructions': parser.get_format_instructions()})
chain = user_query | llm | parser

output = chain.invoke({'query':'I have ordered one Iphone, but it is not delivered yet, work on this. '})

output

KeyError: "Input to ChatPromptTemplate is missing variables {'formatting_instructions'}.  Expected: ['formatting_instructions', 'query'] Received: ['query']\nNote: if you intended {formatting_instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{formatting_instructions}}'.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT "